In [ ]:
!pip install gymnasium[atari]

In [ ]:
!pip install keras
!pip install tensorflow
!pip install --user Pillow
!pip install --user h5py
!pip install --user keras-rl
!pip install gymnasium[atari]
!pip install keras-rl2

In [1]:
# Install numpy version 1.19.5
!pip install numpy==1.19.5

# Install gym version 0.18.3
!pip install gym==0.18.3

# Install keras version 2.1.0
!pip install keras==2.1.0

# Install the latest version of keras-rl2
!pip install keras-rl2

# Install h5py version 2.10.0
!pip install h5py==2.10.0

# Install Pillow version 7.1.2
!pip install Pillow==7.1.2

# Install atari-py version 0.2.6
!pip install atari-py==0.2.6

# Install tensorflow version 2.2
!pip install tensorflow==2.2

# Install Atari environment support for gym
# Note: This will install/upgrade gym to the latest version that supports Atari, 
# so it's recommended to run this command last if you want to keep gym at version 0.18.3
!pip install gym[atari]


In [9]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'


In [12]:
import numpy as np

import gym
import tensorflow as tf


import atari_py

from keras import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# Initialize Gym environment
ENV_NAME = 'Breakout-v0'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

In [ ]:
# Number of actions
nb_actions = env.action_space.n

In [ ]:
# Define the model architecture
input_shape = (84, 84)
window_length = 4




train_model = Sequential()
train_model.add(Flatten(input_shape=(3,) + env.observation_space.shape))
train_model.add(Dense(64, activation='relu'))
train_model.add(Dense(32, activation='relu'))
train_model.add(Dense(16, activation='relu'))
train_model.add(Dense(nb_actions, activation='linear'))



In [ ]:
import keras
import tensorflow as tf

print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000, window_length=window_length)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=1000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Train the agent
dqn.fit(env, nb_steps=1750000, visualize=False, verbose=2)

In [ ]:
# Save the final policy network
dqn.save_weights('policy.h5', overwrite=True)

# Play Environment

In [ ]:
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = GreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Load the trained weights
dqn.load_weights('policy.h5')

# Evaluate the agent
dqn.test(env, nb_episodes=10, visualize=True)